# 16. Interval Kepercayaan (Confidence Intervals)

## Pengenalan Interval Kepercayaan

Interval kepercayaan adalah rentang nilai yang diperkirakan mengandung parameter populasi yang sebenarnya dengan tingkat kepercayaan tertentu. Ini adalah salah satu konsep fundamental dalam statistika inferensial.

### Konsep Dasar

**Confidence Interval (CI)** adalah:
- Rentang nilai yang diperkirakan mengandung parameter populasi
- Dengan tingkat kepercayaan tertentu (biasanya 95%, 99%)
- Berdasarkan data sampel yang diamati

### Jenis-jenis Interval Kepercayaan

1. **CI untuk Mean Populasi**:
   - Menggunakan distribusi normal (n ≥ 30) atau t-distribution (n < 30)
   - Formula: x̄ ± z*(σ/√n) atau x̄ ± t*(s/√n)

2. **CI untuk Proporsi Populasi**:
   - Menggunakan distribusi normal
   - Formula: p̂ ± z*√(p̂(1-p̂)/n)

3. **CI untuk Variance Populasi**:
   - Menggunakan distribusi chi-square
   - Formula: [(n-1)s²/χ²_α/2, (n-1)s²/χ²_1-α/2]

### Interpretasi Interval Kepercayaan

- **95% CI**: Jika kita mengambil 100 sampel dan membuat 100 interval, sekitar 95 interval akan mengandung parameter populasi yang sebenarnya
- **Margin of Error**: Setengah dari lebar interval kepercayaan
- **Sample Size**: Ukuran sampel mempengaruhi lebar interval


In [ ]:
# Import libraries yang diperlukan
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import norm, t, chi2
import warnings
warnings.filterwarnings('ignore')

# Set style untuk visualisasi
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("Libraries berhasil diimport!")


## 1. Interval Kepercayaan untuk Mean Populasi

### CI dengan Distribusi Normal (n ≥ 30)


In [ ]:
def confidence_interval_mean_normal(sample, confidence_level=0.95, pop_std=None):
    """
    Menghitung interval kepercayaan untuk mean populasi menggunakan distribusi normal
    
    Parameters:
    sample: array data sampel
    confidence_level: tingkat kepercayaan (0.95 untuk 95%)
    pop_std: standard deviation populasi (jika diketahui)
    
    Returns:
    dict dengan hasil interval kepercayaan
    """
    n = len(sample)
    sample_mean = np.mean(sample)
    sample_std = np.std(sample, ddof=1)
    
    # Tentukan standard error
    if pop_std is not None:
        se = pop_std / np.sqrt(n)
        distribution = 'normal'
    else:
        se = sample_std / np.sqrt(n)
        distribution = 'normal (estimated σ)'
    
    # Hitung critical value
    alpha = 1 - confidence_level
    z_critical = norm.ppf(1 - alpha/2)
    
    # Hitung margin of error
    margin_error = z_critical * se
    
    # Hitung interval
    ci_lower = sample_mean - margin_error
    ci_upper = sample_mean + margin_error
    
    return {
        'sample_mean': sample_mean,
        'sample_std': sample_std,
        'standard_error': se,
        'confidence_level': confidence_level,
        'z_critical': z_critical,
        'margin_error': margin_error,
        'ci_lower': ci_lower,
        'ci_upper': ci_upper,
        'distribution': distribution
    }

# Contoh data
np.random.seed(42)
sample_data = np.random.normal(50, 10, 100)  # Mean=50, Std=10, n=100

# Hitung 95% CI
ci_95 = confidence_interval_mean_normal(sample_data, confidence_level=0.95)
ci_99 = confidence_interval_mean_normal(sample_data, confidence_level=0.99)

print("=== CONFIDENCE INTERVAL UNTUK MEAN (Normal Distribution) ===")
print(f"Sample size: {len(sample_data)}")
print(f"Sample mean: {ci_95['sample_mean']:.4f}")
print(f"Sample std: {ci_95['sample_std']:.4f}")
print(f"Standard error: {ci_95['standard_error']:.4f}")
print(f"\n95% Confidence Interval:")
print(f"  Lower bound: {ci_95['ci_lower']:.4f}")
print(f"  Upper bound: {ci_95['ci_upper']:.4f}")
print(f"  Margin of error: {ci_95['margin_error']:.4f}")
print(f"\n99% Confidence Interval:")
print(f"  Lower bound: {ci_99['ci_lower']:.4f}")
print(f"  Upper bound: {ci_99['ci_upper']:.4f}")
print(f"  Margin of error: {ci_99['margin_error']:.4f}")


In [ ]:
# Visualisasi Confidence Interval
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Histogram dengan CI
ax1.hist(sample_data, bins=20, density=True, alpha=0.7, color='skyblue', edgecolor='black')
ax1.axvline(ci_95['sample_mean'], color='red', linestyle='-', linewidth=2, label=f'Sample Mean: {ci_95["sample_mean"]:.2f}')
ax1.axvline(ci_95['ci_lower'], color='green', linestyle='--', linewidth=2, label=f'95% CI Lower: {ci_95["ci_lower"]:.2f}')
ax1.axvline(ci_95['ci_upper'], color='green', linestyle='--', linewidth=2, label=f'95% CI Upper: {ci_95["ci_upper"]:.2f}')
ax1.fill_between([ci_95['ci_lower'], ci_95['ci_upper']], 0, ax1.get_ylim()[1], alpha=0.3, color='green', label='95% CI')
ax1.set_xlabel('Value')
ax1.set_ylabel('Density')
ax1.set_title('Sample Distribution with 95% CI')
ax1.legend()

# Perbandingan CI 95% vs 99%
confidence_levels = [0.90, 0.95, 0.99]
ci_widths = []
for cl in confidence_levels:
    ci = confidence_interval_mean_normal(sample_data, confidence_level=cl)
    width = ci['ci_upper'] - ci['ci_lower']
    ci_widths.append(width)

ax2.bar(confidence_levels, ci_widths, color=['lightcoral', 'lightgreen', 'lightblue'], alpha=0.7, edgecolor='black')
ax2.set_xlabel('Confidence Level')
ax2.set_ylabel('CI Width')
ax2.set_title('Confidence Interval Width vs Confidence Level')
ax2.set_xticks(confidence_levels)
ax2.set_xticklabels([f'{int(cl*100)}%' for cl in confidence_levels])

# Add value labels on bars
for i, width in enumerate(ci_widths):
    ax2.text(confidence_levels[i], width + 0.1, f'{width:.2f}', ha='center', va='bottom')

plt.tight_layout()
plt.show()


### CI dengan Distribusi t (n < 30)


In [ ]:
def confidence_interval_mean_t(sample, confidence_level=0.95):
    """
    Menghitung interval kepercayaan untuk mean populasi menggunakan distribusi t
    
    Parameters:
    sample: array data sampel
    confidence_level: tingkat kepercayaan (0.95 untuk 95%)
    
    Returns:
    dict dengan hasil interval kepercayaan
    """
    n = len(sample)
    sample_mean = np.mean(sample)
    sample_std = np.std(sample, ddof=1)
    
    # Hitung standard error
    se = sample_std / np.sqrt(n)
    
    # Hitung degrees of freedom
    df = n - 1
    
    # Hitung critical value
    alpha = 1 - confidence_level
    t_critical = t.ppf(1 - alpha/2, df)
    
    # Hitung margin of error
    margin_error = t_critical * se
    
    # Hitung interval
    ci_lower = sample_mean - margin_error
    ci_upper = sample_mean + margin_error
    
    return {
        'sample_mean': sample_mean,
        'sample_std': sample_std,
        'standard_error': se,
        'confidence_level': confidence_level,
        't_critical': t_critical,
        'degrees_freedom': df,
        'margin_error': margin_error,
        'ci_lower': ci_lower,
        'ci_upper': ci_upper,
        'distribution': 't-distribution'
    }

# Contoh data dengan sampel kecil
np.random.seed(42)
small_sample = np.random.normal(50, 10, 15)  # n=15

# Hitung CI dengan t-distribution
ci_t = confidence_interval_mean_t(small_sample, confidence_level=0.95)

print("=== CONFIDENCE INTERVAL UNTUK MEAN (t-Distribution) ===")
print(f"Sample size: {len(small_sample)}")
print(f"Sample mean: {ci_t['sample_mean']:.4f}")
print(f"Sample std: {ci_t['sample_std']:.4f}")
print(f"Standard error: {ci_t['standard_error']:.4f}")
print(f"Degrees of freedom: {ci_t['degrees_freedom']}")
print(f"t-critical: {ci_t['t_critical']:.4f}")
print(f"\n95% Confidence Interval:")
print(f"  Lower bound: {ci_t['ci_lower']:.4f}")
print(f"  Upper bound: {ci_t['ci_upper']:.4f}")
print(f"  Margin of error: {ci_t['margin_error']:.4f}")

# Perbandingan dengan normal distribution (untuk referensi)
ci_normal = confidence_interval_mean_normal(small_sample, confidence_level=0.95)
print(f"\nPerbandingan dengan Normal Distribution:")
print(f"  t-distribution CI: [{ci_t['ci_lower']:.4f}, {ci_t['ci_upper']:.4f}]")
print(f"  Normal distribution CI: [{ci_normal['ci_lower']:.4f}, {ci_normal['ci_upper']:.4f}]")
print(f"  Perbedaan lebar: {(ci_t['ci_upper'] - ci_t['ci_lower']) - (ci_normal['ci_upper'] - ci_normal['ci_lower']):.4f}")


## 2. Interval Kepercayaan untuk Proporsi Populasi

### CI untuk Proporsi dengan Distribusi Normal


In [ ]:
def confidence_interval_proportion(successes, n, confidence_level=0.95):
    """
    Menghitung interval kepercayaan untuk proporsi populasi
    
    Parameters:
    successes: jumlah keberhasilan
    n: ukuran sampel
    confidence_level: tingkat kepercayaan
    
    Returns:
    dict dengan hasil interval kepercayaan
    """
    # Hitung proporsi sampel
    p_hat = successes / n
    
    # Hitung critical value
    alpha = 1 - confidence_level
    z_critical = norm.ppf(1 - alpha/2)
    
    # Hitung standard error
    se = np.sqrt((p_hat * (1 - p_hat)) / n)
    
    # Hitung margin of error
    margin_error = z_critical * se
    
    # Hitung interval
    ci_lower = max(0, p_hat - margin_error)  # Tidak boleh negatif
    ci_upper = min(1, p_hat + margin_error)  # Tidak boleh lebih dari 1
    
    return {
        'successes': successes,
        'n': n,
        'p_hat': p_hat,
        'confidence_level': confidence_level,
        'z_critical': z_critical,
        'standard_error': se,
        'margin_error': margin_error,
        'ci_lower': ci_lower,
        'ci_upper': ci_upper
    }

# Contoh: Survei kepuasan pelanggan
# Dari 200 pelanggan, 150 menyatakan puas
successes = 150
n = 200

ci_prop = confidence_interval_proportion(successes, n, confidence_level=0.95)

print("=== CONFIDENCE INTERVAL UNTUK PROPORSI ===")
print(f"Sample size: {n}")
print(f"Successes: {successes}")
print(f"Sample proportion: {ci_prop['p_hat']:.4f}")
print(f"Standard error: {ci_prop['standard_error']:.4f}")
print(f"\n95% Confidence Interval:")
print(f"  Lower bound: {ci_prop['ci_lower']:.4f}")
print(f"  Upper bound: {ci_prop['ci_upper']:.4f}")
print(f"  Margin of error: {ci_prop['margin_error']:.4f}")
print(f"\nInterpretasi:")
print(f"  Kita 95% yakin bahwa proporsi populasi yang puas berada antara {ci_prop['ci_lower']:.1%} dan {ci_prop['ci_upper']:.1%}")


## Confidence Interval untuk Proporsi (Proportion)

**Confidence Interval untuk Proporsi** digunakan ketika kita ingin mengestimasi proporsi populasi berdasarkan data sampel. Ini sangat berguna dalam survei, polling, dan analisis data kategorikal.

### Formula:
- **CI = p̂ ± z*√(p̂(1-p̂)/n)**
- **p̂** = proporsi sampel
- **z** = z-score untuk tingkat kepercayaan
- **n** = ukuran sampel

### Kondisi yang Harus Dipenuhi:
1. **n*p̂ ≥ 5** dan **n*(1-p̂) ≥ 5** (untuk pendekatan normal)
2. **Sampel random**
3. **n ≤ 0.05*N** (jika sampling tanpa pengembalian)


In [ ]:
# Confidence Interval untuk Proporsi
def confidence_interval_proportion(successes, n, confidence_level=0.95):
    """
    Menghitung confidence interval untuk proporsi
    
    Parameters:
    successes: Jumlah keberhasilan dalam sampel
    n: Ukuran sampel
    confidence_level: Tingkat kepercayaan (default 0.95)
    
    Returns:
    tuple: (lower_bound, upper_bound, margin_error)
    """
    # Hitung proporsi sampel
    p_hat = successes / n
    
    # Cek kondisi normal approximation
    if n * p_hat < 5 or n * (1 - p_hat) < 5:
        print("Warning: Kondisi normal approximation tidak terpenuhi!")
        print(f"n*p̂ = {n * p_hat:.2f}, n*(1-p̂) = {n * (1 - p_hat):.2f}")
        print("Gunakan metode exact atau binomial confidence interval")
    
    # Hitung z-score
    alpha = 1 - confidence_level
    z_score = norm.ppf(1 - alpha/2)
    
    # Hitung standard error
    se = np.sqrt(p_hat * (1 - p_hat) / n)
    
    # Hitung margin of error
    margin_error = z_score * se
    
    # Hitung confidence interval
    lower_bound = p_hat - margin_error
    upper_bound = p_hat + margin_error
    
    return lower_bound, upper_bound, margin_error

# Contoh 1: Survei kepuasan pelanggan
print("=== CONTOH 1: SURVEI KEPUASAN PELANGGAN ===")
print("Dari 500 pelanggan yang disurvei, 350 menyatakan puas.")
print("Hitung 95% confidence interval untuk proporsi pelanggan yang puas.")

successes = 350
n = 500
confidence_level = 0.95

ci_lower, ci_upper, margin_error = confidence_interval_proportion(successes, n, confidence_level)

print(f"\nHasil:")
print(f"  - Proporsi sampel (p̂): {successes/n:.4f}")
print(f"  - Ukuran sampel (n): {n}")
print(f"  - Standard Error: {np.sqrt(successes/n * (1-successes/n) / n):.4f}")
print(f"  - Margin of Error: {margin_error:.4f}")
print(f"  - 95% CI: [{ci_lower:.4f}, {ci_upper:.4f}]")
print(f"  - Interpretasi: Kita 95% yakin bahwa proporsi pelanggan yang puas berada antara {ci_lower:.1%} dan {ci_upper:.1%}")

# Contoh 2: Survei pemilihan presiden
print("\n=== CONTOH 2: SURVEI PEMILIHAN PRESIDEN ===")
print("Dari 1000 responden, 420 memilih kandidat A.")
print("Hitung 99% confidence interval untuk proporsi pemilih kandidat A.")

successes = 420
n = 1000
confidence_level = 0.99

ci_lower, ci_upper, margin_error = confidence_interval_proportion(successes, n, confidence_level)

print(f"\nHasil:")
print(f"  - Proporsi sampel (p̂): {successes/n:.4f}")
print(f"  - Ukuran sampel (n): {n}")
print(f"  - Standard Error: {np.sqrt(successes/n * (1-successes/n) / n):.4f}")
print(f"  - Margin of Error: {margin_error:.4f}")
print(f"  - 99% CI: [{ci_lower:.4f}, {ci_upper:.4f}]")
print(f"  - Interpretasi: Kita 99% yakin bahwa proporsi pemilih kandidat A berada antara {ci_lower:.1%} dan {ci_upper:.1%}")

# Visualisasi Confidence Interval untuk Proporsi
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Plot 1: Confidence Interval untuk contoh 1
axes[0].errorbar(0, successes/n, yerr=margin_error, fmt='o', capsize=10, capthick=2, 
                 color='blue', markersize=8, label='Sample Proportion ± CI')
axes[0].axhline(successes/n, color='blue', linestyle='-', linewidth=2, alpha=0.7)
axes[0].axhline(ci_lower, color='red', linestyle=':', alpha=0.7, label=f'CI Lower: {ci_lower:.3f}')
axes[0].axhline(ci_upper, color='red', linestyle=':', alpha=0.7, label=f'CI Upper: {ci_upper:.3f}')
axes[0].set_xlim(-0.5, 0.5)
axes[0].set_ylabel('Proportion')
axes[0].set_title('95% CI untuk Proporsi Kepuasan Pelanggan')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Plot 2: Confidence Interval untuk contoh 2
ci_lower2, ci_upper2, margin_error2 = confidence_interval_proportion(420, 1000, 0.99)
axes[1].errorbar(0, 420/1000, yerr=margin_error2, fmt='o', capsize=10, capthick=2, 
                 color='green', markersize=8, label='Sample Proportion ± CI')
axes[1].axhline(420/1000, color='green', linestyle='-', linewidth=2, alpha=0.7)
axes[1].axhline(ci_lower2, color='red', linestyle=':', alpha=0.7, label=f'CI Lower: {ci_lower2:.3f}')
axes[1].axhline(ci_upper2, color='red', linestyle=':', alpha=0.7, label=f'CI Upper: {ci_upper2:.3f}')
axes[1].set_xlim(-0.5, 0.5)
axes[1].set_ylabel('Proportion')
axes[1].set_title('99% CI untuk Proporsi Pemilih Kandidat A')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()
